# Elko

## Инициализация

In [1]:
import os
import sys

from django.utils import timezone

sys.path.append('/home/ubuntu/anodos.ru/anodos/')
os.environ['DJANGO_SETTINGS_MODULE'] = 'anodos.settings'

from django.core.wsgi import get_wsgi_application
application = get_wsgi_application()


import re
import catalog.runner
from catalog.models import *


class Runner(catalog.runner.Runner):

    name  = 'Elko'
    alias = 'elko'
    url = {
        'start'  : 'https://ecom.elko.ru/Account/Login',
        'login'  : 'https://ecom.elko.ru/Account/Login',
        'price'  : 'https://ecom.elko.ru/Catalog/PriceList?'}

    def __init__(self):

        super().__init__()

        self.stock = self.take_stock('stock', 'склад', 3, 10)
        self.transit = self.take_stock('transit', 'транзит', 10, 60)
        
    def run(self):

        # Заходим на начальную страницу
        tree = self.load_html(self.url['start'])
        token = self.xpath_string(tree, './/input[@name="__RequestVerificationToken"]/@value')

        self.login({'__RequestVerificationToken': token,
                    'Amnesia': '',
                    'Username': self.updater.login,
                    'Password': self.updater.password,
                    'submit': 'Войти',
                    'Username2': ''})
        data = self.load(self.url['price'], result_type = 'content')
        
        return data

In [2]:
s = Runner()
data = s.run()

/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is be

In [3]:
import xlrd

# Номера строк и столбцов
num = {'header': 5}

# Распознаваемые слова
word = {'category': 'Категория',
        'category_sub': 'Подкатегория',
        'party_article': 'Код ELKO',
        'product_vendor': 'Производитель',
        'product_article': 'Заводской код',
        'product_name': 'Название и описание продукта',
        'product_description': 'Дополнительная информация',
        'party_price': 'Цена',
        'party_quantity': 'В наличии',
        'product_warranty': 'Гарантия',
        'product_url': 'Ссылка на товар'}

book = xlrd.open_workbook(file_contents = data)

In [4]:
sheet = book.sheet_by_index(0)

In [7]:
for row_num in range(sheet.nrows):
    row = sheet.row_values(row_num)

    # Пустые строки
    if row_num < num['header']:
        continue

    # Заголовок таблицы
    elif row_num == num['header']:
        for cel_num, cel in enumerate(row):
            if   str(cel).strip() == word['category']:
                num['category'] = cel_num
            elif str(cel).strip() == word['category_sub']:
                num['category_sub'] = cel_num
            elif str(cel).strip() == word['party_article']:
                num['party_article'] = cel_num
            elif str(cel).strip() == word['product_vendor']:
                num['product_vendor'] = cel_num
            elif str(cel).strip() == word['product_article']:
                num['product_article'] = cel_num
            elif str(cel).strip() == word['product_name']:
                num['product_name'] = cel_num
            elif str(cel).strip() == word['product_description']:
                num['product_description'] = cel_num
            elif str(cel).strip() == word['party_price']:
                num['party_price'] = cel_num
            elif str(cel).strip() == word['party_quantity']:
                num['party_quantity'] = cel_num
            elif str(cel).strip() == word['product_warranty']:
                num['product_warranty'] = cel_num
            elif str(cel).strip() == word['product_url']:
                num['product_url'] = cel_num

        # Проверяем, все ли столбцы распознались
        if len(num) > len(word):
            print("Структура данных без изменений.")
        else:
            raise(ValueError("Ошибка структуры данных: не все столбцы опознаны."))

    # Товар
    elif row[num['product_article']] and row[num['product_vendor']]:

        product_ = {}
        party_ = {}

        # Категория
        category = "{} | {}".format(row[num['category']], row[num['category_sub']])

        # Производитель
        product_['vendor'] = s.fix_name(row[num['product_vendor']])
        product_['vendor'] = Vendor.objects.get_by_key(updater = s.updater, key = product_['vendor'])
        
        # Продукт
        product_['article'] = s.fix_article(row[num['product_vendor']])
        product_['article'] = s.fix_article(row[num['product_article']])
        product_['name'] = s.fix_name(row[num['product_name']])
        product_['description'] = s.fix_name(row[num['product_description']])
        product_['warranty'] = s.fix_name(row[num['product_warranty']])
        product_['url'] = s.fix_name(row[num['product_url']])

        try:
            product = Product.objects.take(article  = product_['article'],
                                           vendor   = product_['vendor'],
                                           name     = product_['name'])
            #s.products.append(product)
        except ValueError as error:
            continue

        # Партия
        party_['quantity_stock'] = s.fix_quantity(row[num['party_quantity']])
        if row[num['party_quantity']] == 'в транзите':
            party_['quantity_transit'] = None
        else:
            party_['quantity_transit'] = 0

        party_['article'] = s.fix_quantity(row[num['party_article']])
        party_['price'] = s.fix_price(row[num['party_price']])

        try:
            party = Party.objects.make(product = product,
                                       stock = s.stock,
                                       article = party_['article'],
                                       price = party_['price'],
                                       currency = s.rub,
                                       quantity = party_['quantity_stock'],
                                       time = s.start_time)
            #self.parties.append(party)
        except ValueError as error:
            pass

        try:
            party = Party.objects.make(product = product,
                                       stock = s.transit,
                                       article = party_['article'],
                                       price = party_['price'],
                                       currency = s.rub,
                                       quantity = party_['quantity_transit'],
                                       time = s.start_time)
            #self.parties.append(party)
        except ValueError as error:
            pass


Структура данных без изменений.
EZVIZ CS-S5-212WFBS-G
EZVIZ CS-S5-212WFBS-G = 9 890,0 р.; 1 on elko-stock
EZVIZ CS-S5PLUS-212WFBS-B
EZVIZ CS-S5PLUS-212WFBS-B = 18 910,0 р.; 3 on elko-stock
EZVIZ CS-S5PLUS-212WFBS-S
EZVIZ CS-S5PLUS-212WFBS-S = 18 910,0 р.; 2 on elko-stock
HGST 0T00831
HGST 0T00831 = 115 640,0 р.; 10 on elko-stock
HGST 0T00833
HGST 0T00833 = 233 720,0 р.; 1 on elko-stock
Intel SSDPEDMD800G401933089
Intel SSDPEDMD800G401933089 = 120 190,0 р.; None on elko-stock
Intel SSDPEDMD016T401933090
Intel SSDPEDMD016T401933090 = 231 150,0 р.; 4 on elko-stock
Intel SSDPEDMD020T401933091
Intel SSDPEDMD020T401933091 = 277 380,0 р.; 5 on elko-stock
Intel SSDPEDME012T401934677
Intel SSDPEDME012T401934677 = 114 040,0 р.; 1 on elko-stock
Intel SSDPEDME016T401934678
Intel SSDPEDME016T401934678 = 141 770,0 р.; 1 on elko-stock
Intel SSDPEDMX400G401937525
Intel SSDPEDMW012T4X1944777
Intel SSDPEDMW012T4X1944777 = 74 020,0 р.; 7 on elko-stock
Intel SSDPEDMW400G4X1944775
Intel SSDPEDMW400G4X19447

Silicon Power SP120GBSS3S60S25
Silicon Power SP120GBSS3S60S25 = 4 160,0 р.; None on elko-stock
Silicon Power SP060GBSS3S60S25
Silicon Power SP060GBSS3S60S25 = 2 980,0 р.; 100 on elko-stock
Silicon Power SP240GBSS3S70S25
Silicon Power SP240GBSS3S70S25 = 6 510,0 р.; 10 on elko-stock
Silicon Power SP480GBSS3S70S25
Silicon Power SP480GBSS3S70S25 = 11 590,0 р.; 10 on elko-stock
Silicon Power SP120GBSS3V70S25
Silicon Power SP120GBSS3V70S25 = 4 340,0 р.; 10 on elko-stock
Toshiba THN-S101Z2400E8
Toshiba THN-S101Z2400E8 = 5 820,0 р.; None on elko-stock
Western Digital WDS100T1B0A
Western Digital WDS100T1B0A = 18 910,0 р.; 8 on elko-stock
Western Digital WDS500G1B0A
Western Digital WDS500G1B0A = 11 130,0 р.; None on elko-stock
Western Digital WDS250G1B0A
Western Digital WDS250G1B0A = 6 080,0 р.; None on elko-stock
Western Digital WDS240G1G0A
Western Digital WDS240G1G0A = 5 710,0 р.; 100 on elko-stock
Western Digital WDS120G1G0A
Western Digital WDS120G1G0A = 4 920,0 р.; None on elko-stock
Cooler 

Gigabyte GV-RX550D5-2GD
Gigabyte GV-RX550D5-2GD = 5 770,0 р.; 40 on elko-stock
Gigabyte GV-N1060AORUS-6GD
Gigabyte GV-N1060AORUS-6GD = 27 870,0 р.; None on elko-stock
Gigabyte GV-RX570AORUS-4GD
Gigabyte GV-RX570AORUS-4GD = 24 420,0 р.; None on elko-stock
Gigabyte GV-N1060AORUSX-6GD
Gigabyte GV-N1060AORUSX-6GD = 28 310,0 р.; None on elko-stock
Gigabyte GV-N1030D5-2GL
Gigabyte GV-N1030D5-2GL = 4 700,0 р.; 100 on elko-stock
Gigabyte GV-RX560GAMINGOC-4GD
Gigabyte GV-RX560GAMINGOC-4GD = 9 770,0 р.; None on elko-stock
Gigabyte GV-N1030SL-2GL
Gigabyte GV-N1030SL-2GL = 4 800,0 р.; 40 on elko-stock
Gigabyte GV-RX560GAMINGOC-2GD
Gigabyte GV-RX560GAMINGOC-2GD = 8 090,0 р.; 10 on elko-stock
Gigabyte GV-N1060G1GAMING-3GD2.0
Gigabyte GV-N1060G1GAMING-3GD2.0 = 22 860,0 р.; None on elko-stock
Gigabyte GV-N1030OC-2GI
Gigabyte GV-N1030OC-2GI = 5 080,0 р.; None on elko-stock
Gigabyte GV-N105TOC-4GDV1.1
Gigabyte GV-N105TOC-4GDV1.1 = 10 650,0 р.; None on elko-stock
Gigabyte GV-N108TAORUSXWB-11GD
Gigabyte G

HGST 0S04005 = 8 770,0 р.; 100 on elko-stock
HGST 0S04007
HGST 0S04007 = 13 320,0 р.; 40 on elko-stock
Seagate ST3000DM008
Seagate ST3000DM008 = 6 160,0 р.; 100 on elko-stock
Seagate ST1000DX002
Seagate ST1000DX002 = 5 240,0 р.; 100 on elko-stock
Seagate ST2000DM006
Seagate ST2000DM006 = 4 440,0 р.; 100 on elko-stock
Seagate ST500DM009
Seagate ST500DM009 = 2 780,0 р.; 100 on elko-stock
Seagate ST10000VN0004
Seagate ST10000VN0004 = 24 310,0 р.; None on elko-stock
Seagate ST10000VX0004
Seagate ST10000VX0004 = 25 710,0 р.; 40 on elko-stock
Seagate ST10000DM0004
Seagate ST10000DM0004 = 25 780,0 р.; 10 on elko-stock
Seagate ST1000VN002
Seagate ST1000VN002 = 3 860,0 р.; 100 on elko-stock
Seagate ST1000VX005
Seagate ST1000VX005 = 3 650,0 р.; 40 on elko-stock
Seagate ST2000VN004
Seagate ST2000VN004 = 5 330,0 р.; 3 on elko-stock
Seagate ST2000VX008
Seagate ST2000VX008 = 5 240,0 р.; 100 on elko-stock
Seagate ST3000VN007
Seagate ST3000VN007 = 6 620,0 р.; 40 on elko-stock
Seagate ST4000VX007
Seaga

In Win EAR002BSS450HQ70
In Win EAR002BSS450HQ70 = 3 830,0 р.; 5 on elko-stock
In Win ES861BK
In Win ES861BK = 2 120,0 р.; 10 on elko-stock
In Win ES862BK
In Win ES862BK = 2 190,0 р.; 40 on elko-stock
In Win ES722BK
In Win ES722BK = 2 060,0 р.; None on elko-stock
In Win EAR003BSS450HQ70
In Win EAR003BSS450HQ70 = 3 760,0 р.; 40 on elko-stock
In Win EL501
In Win EL501 = 2 640,0 р.; 100 on elko-stock
In Win EC021BLRB-S450T70
In Win EC021BLRB-S450T70 = 3 390,0 р.; 10 on elko-stock
In Win EMR035
In Win EMR035 = 3 600,0 р.; 10 on elko-stock
In Win MG137BL
In Win MG137BL = 6 290,0 р.; 10 on elko-stock
In Win EQ101PM-200ATXPROAPFC
In Win EQ101PM-200ATXPROAPFC = 2 440,0 р.; None on elko-stock
In Win EFS052RB-S500HQ70
In Win EFS052RB-S500HQ70 = 3 660,0 р.; 10 on elko-stock
In Win EFS057RB-S500HQ70
In Win EFS057RB-S500HQ70 = 3 910,0 р.; 40 on elko-stock
In Win EMR048
In Win EMR048 = 3 840,0 р.; 10 on elko-stock
In Win ES701
In Win ES701 = 2 430,0 р.; 10 on elko-stock
In Win ES726
In Win ES726 = 2 

...

Ubiquiti ES-24-250W
Ubiquiti ES-24-250W = 24 960,0 р.; 6 on elko-stock
Ubiquiti US-24-250W
Ubiquiti US-24-250W = 24 620,0 р.; None on elko-stock
Ubiquiti ES-16-150W
Ubiquiti ES-16-150W = 18 630,0 р.; None on elko-stock
Ubiquiti R5AC-LITE
Ubiquiti R5AC-LITE = 7 790,0 р.; 5 on elko-stock
Ubiquiti R5AC-PRISM
Ubiquiti R5AC-PRISM = 10 750,0 р.; 40 on elko-stock
Huawei 51068685
Huawei 51068685 = 5 410,0 р.; 100 on elko-stock
Huawei 51069014
Huawei 51069014 = 5 910,0 р.; 100 on elko-stock
Netis MW5230
Netis MW5230 = 1 008,0 р.; 100 on elko-stock
D-Link DEM-311GT/G1A
D-Link DEM-310GT/G1A
D-Link DEM-310GT/G1A = 1 392,0 р.; 10 on elko-stock
Huawei 02315204
Huawei 02315204 = 4 320,0 р.; 10 on elko-stock
Huawei 02318169
Huawei 02318169 = 18 220,0 р.; 10 on elko-stock
Huawei 02317346
Huawei 02317346 = 27 220,0 р.; 10 on elko-stock
Huawei 02315200
Huawei 02315200 = 8 410,0 р.; 40 on elko-stock
LSI OC10-SR-OPT-2
LSI OC10-SR-OPT-2 = 23 470,0 р.; 9 on elko-stock
Mikrotik S-85DLC05D
Mikrotik S-85DLC05D 

Intel I210T1BLK921434
Intel I210T1BLK921434 = 4 740,0 р.; 10 on elko-stock
Intel E10G42BTDABLK927249
Intel E10G42BTDABLK927249 = 19 650,0 р.; 40 on elko-stock
Intel I350T2V2BLK936714
Intel I350T2V2BLK936714 = 7 800,0 р.; None on elko-stock
Intel I350T4V2BLK936716
Intel I350T4V2BLK936716 = 19 960,0 р.; 100 on elko-stock
Intel X710DA2BLK933217
Intel X710DA2BLK933217 = 21 010,0 р.; 8 on elko-stock
Intel X710DA4FHBLK932576
Intel X710DA4FHBLK932576 = 28 800,0 р.; 10 on elko-stock
Intel XL710QDA2BLK932587
Intel XL710QDA2BLK932587 = 33 880,0 р.; 7 on elko-stock
Intel X550T2BLK940136
Intel X550T2BLK940136 = 27 100,0 р.; 10 on elko-stock
Netis AD1103
Netis AD1103 = 450,0 р.; 100 on elko-stock
Synology E10G15-F1
Synology E10G15-F1 = 12 270,0 р.; None on elko-stock
Synology E10G17-F2
Synology E10G17-F2 = 20 960,0 р.; 3 on elko-stock
HIPER Power NT-UTP5EMM18
HIPER Power NT-UTP5EMM18 = 58,5 р.; 100 on elko-stock
HIPER Power NT-UTP5EMM30
HIPER Power NT-UTP5EMM30 = 64,5 р.; 100 on elko-stock
HIPER Po

Synology VS360HD
Synology VS360HD = 36 740,0 р.; 1 on elko-stock
Synology NVR2164CH
Synology NVR2164CH = 32 470,0 р.; 1 on elko-stock
Synology NVR2169CH
Synology NVR2169CH = 44 670,0 р.; 2 on elko-stock
Dot Hill PFRUKE08-01
Dot Hill PFRUKE08-01 = 1 018,0 р.; 2 on elko-stock
HGST 1EX0081
HGST 1EX0081 = 114 320,0 р.; 4 on elko-stock
Huawei 02131042
Huawei 02131042 = 6 220,0 р.; 4 on elko-stock
Huawei 2130957
Huawei 2130957 = 10 940,0 р.; 1 on elko-stock
Huawei 02131058
Huawei 02131058 = 13 290,0 р.; 4 on elko-stock
Infortrend 9571CPSU-0010
Infortrend 9571CPSU-0010 = 32 790,0 р.; 1 on elko-stock
Infortrend N3012PSU-0010
Infortrend N3012PSU-0010 = 21 600,0 р.; 1 on elko-stock
Dot Hill D3120C000000DA
Dot Hill D3120C000000DA = 33 940,0 р.; 1 on elko-stock
HGST 1ES0054
HGST 1ES0054 = 991 840,0 р.; 2 on elko-stock
HGST 1ES0058
HGST 1ES0058 = 1 151 030,0 р.; 1 on elko-stock
HGST 1ES0093
HGST 1ES0093 = 2 002 050,0 р.; 1 on elko-stock
Huawei 02350SKV
Huawei 02350SKV = 141 820,0 р.; 1 on elko-stoc

NetApp 49294-00
NetApp 49294-00 = 476,0 р.; 2 on elko-stock
QLogic XPAK-COPP-03
QLogic XPAK-COPP-03 = 21 440,0 р.; 1 on elko-stock
QLogic XPAK-COPP-09
QLogic XPAK-COPP-09 = 19 920,0 р.; 3 on elko-stock
QNAP RAIL-C01
QNAP RAIL-C01 = 6 200,0 р.; 5 on elko-stock
QNAP RAIL-A01-35
QNAP RAIL-A01-35 = 13 630,0 р.; 4 on elko-stock
QNAP RAIL-B02
QNAP RAIL-B02 = 7 930,0 р.; 6 on elko-stock
QNAP RAIL-A03-57
QNAP RAIL-A03-57 = 17 350,0 р.; 7 on elko-stock
Huawei 02359806
Huawei 02359806 = 112 950,0 р.; None on elko-stock
Huawei 0235G7R9-FC8
Huawei 0235G7R9-FC8 = 189 960,0 р.; 1 on elko-stock
Huawei 02350BVB-10GB-SW
Huawei 02350BVB-10GB-SW = 630 390,0 р.; 1 on elko-stock
Huawei 02350BSE-FC16-SW
Huawei 02350BSE-FC16-SW = 719 090,0 р.; 2 on elko-stock
Huawei 2350SHW-12X4TB-8X8FC
Huawei 2350SHW-12X4TB-8X8FC = 704 580,0 р.; 1 on elko-stock
Huawei 2350SHW-12X2TB-8X8FC
Huawei 2350SHW-12X2TB-8X8FC = 566 420,0 р.; 1 on elko-stock
Huawei 02350SHR-8XFC8-12X1.2TB
Huawei 02350SHR-8XFC8-12X1.2TB = 704 580,0 р.;

Infortrend JB224RB0-8732 = 67 870,0 р.; 2 on elko-stock
Infortrend JB2012R01-8732
Infortrend JB2012R01-8732 = 170 290,0 р.; 3 on elko-stock
Infortrend DS1024R0CB00B-8732
Infortrend DS1024R0CB00B-8732 = 330 160,0 р.; 7 on elko-stock
Infortrend JB2016R01-8732
Infortrend JB2016R01-8732 = 198 720,0 р.; None on elko-stock
Infortrend DS1012R0C000B-8732
Infortrend DS1012R0C000B-8732 = 259 240,0 р.; 9 on elko-stock
Infortrend JB3016R00-8732
Infortrend JB3016R00-8732 = 247 990,0 р.; None on elko-stock
Infortrend GS3012R0C0F0D-8732
Infortrend GS3012R0C0F0D-8732 = 556 500,0 р.; 1 on elko-stock
Infortrend JB3024RB0-8732
Infortrend JB3024RB0-8732 = 286 900,0 р.; 2 on elko-stock
Infortrend GS3024R0CBF0D-8732
Infortrend GS3024R0CBF0D-8732 = 712 600,0 р.; 1 on elko-stock
Infortrend JB2024RB1-8732
Infortrend JB2024RB1-8732 = 240 250,0 р.; 1 on elko-stock
Infortrend JB3060RL00-8732
Infortrend JB3060RL00-8732 = 698 350,0 р.; 1 on elko-stock
Infortrend GS2024R0CBF0D-8732
Infortrend GS2024R0CBF0D-8732 = 50